# MINI-"KAGGEL" COMPETITION WITHIN The Bridge
Using the CIFAR-10 dataset and CNN and DNN let's try to get the maximun accuracy!! Leavin the test_data UNTOUCH is the only rule and we have 24 hours. Let's CODE!!

In [1]:
import keras
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras.applications import VGG19
from keras import models, layers, optimizers
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
#load of the dataset
(train_X, train_y), (test_X, test_y) = cifar10.load_data() 

#normalization of data from dataset
train_X_norm = train_X.astype('float32')/ 255.0 
test_X_norm = test_X.astype('float32')/ 255.0

#we transform the labels to categoricals so we can work better with them
train_y = to_categorical(train_y) 
test_y = to_categorical(test_y)

In [3]:
#import of VGG19 model
net = VGG19(weights = 'imagenet', 
            include_top = False, 
            input_shape = (32, 32, 3),)

In [4]:
#now we'll generate more images with the function ImageDataGenerator() [extracted from: https://keras.io/api/preprocessing/image/]
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(train_X_norm)

In [5]:
model4 = models.Sequential()
model4.add(net)
model4.add(layers.Flatten())
model4.add(layers.Dense(100, activation = 'relu'))
model4.add(layers.Dense(10, activation='softmax'))

In [6]:
model4.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 1, 1, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               51300     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 20,076,694
Trainable params: 20,076,694
Non-trainable params: 0
_________________________________________________________________


In [7]:
model4.compile(optimizer=optimizers.RMSprop(lr = 2e-5),
              loss = 'categorical_crossentropy',
              metrics = ['acc'])

In [8]:
model4.fit(datagen.flow(train_X_norm, train_y, batch_size=32), epochs = 5, validation_data = (test_X_norm, test_y))

Epoch 1/5
1563/1563 [==============================] - 152s 97ms/step - loss: 1.0844 - acc: 0.6204 - val_loss: 1.4402 - val_acc: 0.5250
Epoch 2/5
1563/1563 [==============================] - 149s 95ms/step - loss: 0.7772 - acc: 0.7308 - val_loss: 1.2030 - val_acc: 0.5983
Epoch 3/5
1563/1563 [==============================] - 148s 95ms/step - loss: 0.6781 - acc: 0.7660 - val_loss: 1.4393 - val_acc: 0.5230
Epoch 4/5
1563/1563 [==============================] - 148s 95ms/step - loss: 0.6190 - acc: 0.7862 - val_loss: 1.2702 - val_acc: 0.5795
Epoch 5/5
1563/1563 [==============================] - 148s 95ms/step - loss: 0.5766 - acc: 0.8007 - val_loss: 1.5853 - val_acc: 0.5296


In [9]:
work = VGG19(weights = 'imagenet', 
            include_top = False, 
            input_shape = (32, 32, 3),)

In [11]:
model5 = models.Sequential()
model5.add(work)
model5.add(layers.Flatten())
model5.add(layers.Dropout(0.2))
model5.add(layers.Dense(256, activation = 'relu'))
model5.add(layers.Dense(100, activation = 'relu'))
model5.add(layers.Dense(10, activation='softmax'))

In [12]:
#we unfrozen block5 of the CNN for do the training with the data from the CIFAR-10 dataset and update the weights of the model with forward/backward propagation
work.trainable = True
set_trainable = False
for layer in work.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [13]:
model5.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 1, 1, 512)         20024384  
_________________________________________________________________
flatten_3 (Flatten)          (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 100)               25700     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 20,182,422
Trainable params: 9,597,270
Non-trainable params: 10,585,152
__________________________________

In [14]:
model5.compile(loss = 'categorical_crossentropy',
              optimizer = "rmsprop",
              metrics = ['acc'])

hystory2 = model5.fit(train_X_norm, train_y, epochs = 5, batch_size = 100, validation_data = (test_X_norm, test_y))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 40s 805us/step - loss: 1.6349 - acc: 0.4417 - val_loss: 1.1135 - val_acc: 0.6365
Epoch 2/5
50000/50000 [==============================] - 39s 783us/step - loss: 1.0725 - acc: 0.6488 - val_loss: 0.9915 - val_acc: 0.6605
Epoch 3/5
50000/50000 [==============================] - 39s 784us/step - loss: 0.9549 - acc: 0.6877 - val_loss: 0.9876 - val_acc: 0.6727
Epoch 4/5
50000/50000 [==============================] - 39s 785us/step - loss: 0.8925 - acc: 0.7093 - val_loss: 0.9232 - val_acc: 0.7123
Epoch 5/5
50000/50000 [==============================] - 39s 784us/step - loss: 0.8314 - acc: 0.7305 - val_loss: 0.9308 - val_acc: 0.7056


In [18]:
model6 = models.Sequential()
model6.add(work)
model6.add(layers.Flatten())
model6.add(layers.Dropout(0.2))
model6.add(layers.Dense(256, activation = 'relu'))
model6.add(layers.Dropout(0.2))
model6.add(layers.Dense(100, activation = 'relu'))
model6.add(layers.Dense(10, activation='softmax'))

model6.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 1, 1, 512)         20024384  
_________________________________________________________________
flatten_6 (Flatten)          (None, 512)               0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 100)               25700     
_________________________________________________________________
dense_15 (Dense)             (None, 10)               

In [ ]:
#we unfrozen block5 of the CNN for do the training with the data from the CIFAR-10 dataset and update the weights of the model with forward/backward propagation
work.trainable = True
set_trainable = False
for layer in work.layers:
    if layer.name == 'block4_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [19]:
model6.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 1, 1, 512)         20024384  
_________________________________________________________________
flatten_6 (Flatten)          (None, 512)               0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 100)               25700     
_________________________________________________________________
dense_15 (Dense)             (None, 10)               

In [20]:
model6.compile(loss = 'categorical_crossentropy',
              optimizer = "rmsprop",
              metrics = ['acc'])

hystory2 = model6.fit(train_X_norm, train_y, epochs = 10, batch_size = 100, validation_data = (test_X_norm, test_y))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 40s 793us/step - loss: 0.7673 - acc: 0.8006 - val_loss: 1.1013 - val_acc: 0.7206
Epoch 2/10
50000/50000 [==============================] - 39s 782us/step - loss: 0.5604 - acc: 0.8340 - val_loss: 1.0687 - val_acc: 0.7335
Epoch 3/10
50000/50000 [==============================] - 39s 786us/step - loss: 0.5456 - acc: 0.8381 - val_loss: 1.1755 - val_acc: 0.7304
Epoch 4/10
50000/50000 [==============================] - 39s 787us/step - loss: 0.5623 - acc: 0.8442 - val_loss: 1.5580 - val_acc: 0.7493
Epoch 5/10
50000/50000 [==============================] - 39s 789us/step - loss: 0.5705 - acc: 0.8484 - val_loss: 1.6236 - val_acc: 0.7324
Epoch 6/10
50000/50000 [==============================] - 39s 787us/step - loss: 0.5535 - acc: 0.8531 - val_loss: 1.7244 - val_acc: 0.7250
Epoch 7/10
50000/50000 [==============================] - 39s 787us/step - loss: 0.5487 - acc: 0.8511 - val_loss: 1.